In [4]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import lammps_logfile as lmplog
import os
import glob
import plumed
from lmp_spc_scripts import read_xyz_traj,xyz_cords_array,lmp_data_subs_coord,change_data_file_name

import natsort

import warnings
warnings.filterwarnings("ignore")

For tutoriL purposes we will only work with the lmp_spc_500 folders


In [ ]:
# Name of relevant files and folders
data_folder = '../../OnlyMinima/production_runs/'
# workig_folder = 'umbrella_'
log_file = 'adp_clmd_2ns_umb_' #'log.lammps'
forces_file = 'forces.dump'
xyz_traj_file = 'traj_nnip.xyz'
lmp_spc_folder = './lmp_spc_500Frames/' # Folder were new data files will be saved. LAMMPS spc
workig_folder = 'umbrella_'
og_data_file = './example.input'
lmp_input_file = './input_0.inp'
verbose =True
indices_file = 'index_pcnt_diff.txt'

In [ ]:
# Uncomment for usage 
# os.system('python run_cv_config_select_1.py')


In [6]:
# Load the phi and psi values from the file
# selected_phi_psi = np.loadtxt('selected_phi_psi.txt')

# Load npy file
selected_phi_psi = np.load('selected_phi_psi.npy')
print(selected_phi_psi.shape)
umbrella = selected_phi_psi[:,0]
index = selected_phi_psi[:,1]
phi = selected_phi_psi[:,2]
psi = selected_phi_psi[:,3]

## IMPORTANT NOTE: This values represent the the index (frame) per umbrella where the phi-psi combination are closer to the desired value


(50176, 4)


In [7]:
# Convert to pandas dataframe
import pandas as pd
selected_phi_psi = pd.DataFrame(selected_phi_psi, columns=['umbrella', 'index', 'phi', 'psi'])
#selected_phi_psi.head()

# Delete the rows that have 0.0 in the index,phi,psi columns
selected_phi_psi = selected_phi_psi.loc[selected_phi_psi['index'] != 0.0]


Once the indices have been selected, we can proceed to select the regions in which we want to take samples



In [ ]:
## Selection of points

phi_values =  np.rad2deg(selected_phi_psi['phi']) # Replace with actual phi values
psi_values =    np.rad2deg(selected_phi_psi['psi']) # Replace with actual psi values


user_values_list = [
    (-143.35443037974684, 155.47320410490306),
(-84.49367088607596, 135.63283922462938),
(-80.69620253164558, 59.00798175598629),
(-4.113924050632903, 30.273660205245136),
(59.49367088607596, -39.167616875712696),
(126.2658227848101, 63.45496009122006),
(133.86075949367086, -95.6100342075257),
(-14.87341772151899, -153.76282782212093),
(-148.1012658227848, -99.714937286203),
(-4.74683544303798, -50.11402508551885),
(7.594936708860729, 78.5062713797035),
(64.55696202531647, 110.6613454960091),
(135.12658227848095, -43.61459521094642),
(122.78481012658222, 164.709236031927),
(-144.62025316455697, -31.64196123147093),
(-144.62025316455697, -174.97149372862032)
]  # Add more values as needed

print(len(user_values_list))

# Number of points to select
m = 32  # for 500 frames #125 #250 #625


In [ ]:
## Get euclidean distance between available points and selected centers
indexes =[]

for user_values in user_values_list[:]:
    user_phi, user_psi = user_values
    print(f"User input. phi = {user_phi}, psi = {user_psi}" )

    # Calculate the distance between the user-provided phi and psi values and the phi and psi values in the dataset
    distances = np.sqrt((phi_values - user_phi)**2 + (psi_values - user_psi)**2)
    print(f"Shape of distances array {distances.shape}")

    # Get indices of m closest points
    indices = np.argsort(distances)[:m] 
    print(f"Shape of indices array {indices.shape}")
    print(indices)

    # append indices to list
    indexes.append(indices)

# Convert list to numpy array
indexes = np.array(indexes).T
#reshape array to 1D
indexes = indexes.reshape(-1)
print(f"Shape of indexes array {indexes.shape}")


In [ ]:
## determine the samples near the selected points 

euclid_selected_phi_psi = selected_phi_psi.iloc[indexes]
euclid_selected_phi_psi.head()

In [9]:
##  Code to select coordiantes of configurations and create files for lammps spc 

# grab the values of the umbrella and index columns in arrays 
umbrellas = euclid_selected_phi_psi['umbrella'].values
index = euclid_selected_phi_psi['index'].values


In [ ]:
## Create files for LAMMPs single opint calculations

# Create folder to store the new data files

## Check if the lmp_spc_folder exits, if not create it
if not os.path.exists(lmp_spc_folder):
    os.makedirs(lmp_spc_folder)

    #i = 1

# Loop over indices from selected 
for i,umbrella in enumerate(umbrellas[:5]):
    print(f'\n\nProcessing Umbrella: {int(umbrella)}')

    # 1. Determine the working directory and the index of interest for each umbrella.
    working_dir = f'{data_folder}umbrella_{int(umbrella)}/'
    print(f'Working directory: {working_dir}')
    print(f'Index with smallest error is {int(index[i])}')

    ## 2. Determine the working xyz file.
    working_xyz_file = working_dir + xyz_traj_file
    print(f'Working xyz file: {working_xyz_file}')

    # Read the coordinates from the xyz file
    xyz_traj_dict = read_xyz_traj(working_xyz_file)

    # Extract the coordinates from the dictionary
    xyz_coordinates = xyz_cords_array(xyz_traj_dict,int(index[i]))   #xyz_traj_dict['frames'][indices[i]]

    # Substitute the coordinates in the lammps data file
    lmp_data_subs_coord(xyz_coordinates,og_data_file,f'{lmp_spc_folder}frame_{int(umbrella)}.data',LOUD=False)
    # REname the filename in the read_data command in the lammps input file
    change_data_file_name(f'{lmp_input_file}',f'frame_{int(umbrella)}.data',LOUD=False)

    #3. Create directory to store lammps data and input files.
    os.makedirs(lmp_spc_folder+f'umbrella_{int(umbrella)}', exist_ok=True)

    print(f'Created directory: {lmp_spc_folder}umbrella_{int(umbrella)}')

    # move the data file to the folder
    os.system(f'mv {lmp_spc_folder}frame_{int(umbrella)}.data {lmp_spc_folder}/umbrella_{int(umbrella)}/')

    # Copy the input file to the folder
    os.system(f'cp {lmp_input_file} ./plumed.dat {lmp_spc_folder}/umbrella_{int(umbrella)}/')

print(f'\n\nAll Done! Thank you!!\n\n')



NOw we can extract the Potential energy and atomic forces from the lammps single points


In [10]:
## Redefine the names of files and directores

data_folder = './lmp_spc_500Frames/'
workig_folder = 'umbrella_'
log_file = 'log.lammps' ##'adp_clmd_2ns_umb_'
forces_file = 'forces.dump'
xyz_traj_file = 'traj_nnip.xyz'




In [ ]:
# TEst for one folder only. 

# Initialize data stofing lists
potEng = []
types = []
coordinates = []
forces = []
cv = []
verbose = True


print(f'Working on {data_folder}')
#Loop over the folders inside the data_folder
for i, umbrella in enumerate(natsort.natsorted(os.listdir(data_folder))[:]):
    if umbrella.startswith(workig_folder): # Enter onfly files that start with umbrella
        if verbose:
            print(f'\n{i}. Umbrella Folder: {umbrella}')
            
        # for config in os.listdir(f'{data_folder}{umbrella}'):
        #     if config.startswith('config_'):
        #         print(f'Config Folder: {config}')
        

        
        try: #Try block for log file and potential energy
            # Lod the log file and get the potential energy. THen append it to the list
            log = lmplog.File(f'{data_folder}{umbrella}/{log_file}')
            potEng.append(log.get('PotEng')[0])

            if verbose:
                print(f'{data_folder}{umbrella}/{log_file}{i}.log file found.')
                #print(f'Number of frames: {len(log.get("PotEng"))}')
            
        except:
            print(f'No log file found for {data_folder}{umbrella}\n')
            break
        
            
        try: #Try block for coordinate araray. Extract coords from xyz file 
            xyz_traj_dict = read_xyz_traj(f'{data_folder}{umbrella}/{xyz_traj_file}')
            xyz_coordinates = xyz_cords_array(xyz_traj_dict,0)
            coordinates.append(xyz_coordinates)
            
            if verbose:
                print(f'{data_folder}{umbrella}/{xyz_traj_file} file found.')
                #print(f'Number of frames: {len(xyz_traj_dict["frames"])}')
            
        except:
            print(f'No xyz file found for {data_folder}{umbrella}\n')
            break
            
        try: #Block for forces
            forces_dict = parse_lammpstrj(f'{data_folder}{umbrella}/{forces_file}')
            
            # Sort the dictionary by ID, so every frmae is in the same order
            for key in forces_dict.keys():
                forces_dict[key]['atoms'].sort(key=lambda x: x[0])
                
            forces_temp = [] # Iniftialize list to store forces of the cong
            for row in forces_dict[0]['atoms'][:]:
                forces_temp.append(row[-3:])
            
            #Append to genral forces list    
            forces.append(forces_temp)
            if verbose:
                print(f'{data_folder}{umbrella}/{forces_file} forces file found.')
                #print(f'Number of frames: {len(forces_dict)}')
        except:
            print(f'No forces file found for {data_folder}{umbrella}/\n')
            break

        try:
            colvar = np.loadtxt(f'{data_folder}{umbrella}/colvar_multi_0.dat')
            print(f'{data_folder}{umbrella}/colvar_multi_0.dat Colvar file found')
            cv.append(colvar[0])
        except:
            print(f'No colvar file found for {data_folder}{umbrella}/colvar_multi_0.dat\n')
            break        
            
        
    
for atom in xyz_traj_dict['frames'][0]['atoms_data']:
    #types.append(atom['atom_type'])
    #If atom_type =1, 6 or 7 is an Hydrgen atom so append 1
    if (atom['atom_type'] == 1) or (atom['atom_type'] == 6) or (atom['atom_type'] == 7):
        types.append(1)
    #If atom_type =2 or 3 is an Carbon atom so append 6
    elif (atom['atom_type'] == 2) or (atom['atom_type'] == 3):
        types.append(6)
        
    #If atom_type =4  is an Oxygen atom so append 8
    elif (atom['atom_type'] == 4):
        types.append(8)
        
    #If atom_type =5  is an Nitrogen atom so append 7
    elif (atom['atom_type'] == 5):
        types.append(7)
    

#Define type array for the Alanine Dipeptide molecuele. MAke sure follows the same order as the xyz file and the forces file
# type_array = np.array([1,2,1,1,3,4,5,6,2,7,2,1,1,1,3,4,5,6,2,7,7,7])
type_array = np.array(types)
cv = np.array(cv)
# COnvert list into np arrays
potEng = np.array(potEng)
coordinates = np.array(coordinates)
forces = np.array(forces) 



print(f'\nSummary of data:') 
print(f'\nShape of Potential Energy: {np.shape(potEng)}')
print(f'Shape of Coordinates: {np.shape(coordinates)}')
print(f'Shape of Forces: {np.shape(forces)}')
print(f'Shape of Colvar: {np.shape(cv)}')

print(f'Shape of Type Array: {np.shape(type_array)}')

# #Save to npz file
# np.savez(f'{data_folder}alanine_univ2D_CLC_2500frames_373K.npz', E=potEng, R=coordinates, F=forces, z=type_array)

